In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql.functions import *


In [0]:
df = spark.read.format("parquet").load("abfss://bronze@azuresadatalake.dfs.core.windows.net/products")

df.display()

In [0]:
df = df.drop("_rescued_data")

In [0]:
df.createOrReplaceTempView("products")

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_catalog.bronze.get_discounted_price(p_price DOUBLE)
RETURNS DOUBLE
LANGUAGE SQL
RETURN p_price * 0.9

In [0]:
%sql
select product_id, price, databricks_catalog.bronze.get_discounted_price(price) as discounted_price from products

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_catalog.bronze.upper_func(p_brand STRING)
RETURNS STRING
LANGUAGE PYTHON
AS 
$$
  return p_brand.upper()
$$

In [0]:
%sql

SELECT product_id, brand, databricks_catalog.bronze.upper_func(brand) from products

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@azuresadatalake.dfs.core.windows.net/products")

In [0]:
%sql

create table databricks_catalog.silver.products_silver using delta 
location "abfss://silver@azuresadatalake.dfs.core.windows.net/products"